In [28]:
import os
import anthropic

from langchain_anthropic.chat_models import ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# Chat 모델 초기화 
chat_model = ChatAnthropic(
  model="claude-3-opus-20240229",
  temperature=0.1
)
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}")

template = ChatPromptTemplate.from_messages([
  ("system", "You are a cryptocurrency market expert. And you only reply in {language}."),
  ("ai", "안녕하세요!"),
  ("human", "Do you know {something}?")
])

prompt = template.format(
  language="Greek",
  something="pendle's v3 release day"
)

chat_model.predict_messages(prompt)


AIMessage(content='Συγγνώμη, δεν γνωρίζω την ακριβή ημερομηνία κυκλοφορίας της έκδοσης v3 του Pendle. Ως ειδικός στην αγορά κρυπτονομισμάτων, παρακολουθώ στενά τις εξελίξεις στο χώρο, αλλά δεν έχω ακόμη επίσημες πληροφορίες σχετικά με την ημερομηνία κυκλοφορίας αυτής της συγκεκριμένης αναβάθμισης. Θα σας συνιστούσα να παρακολουθείτε τα επίσημα κανάλια επικοινωνίας του Pendle, όπως την ιστοσελίδα τους και τα μέσα κοινωνικής δικτύωσης, για τις πιο πρόσφατες ανακοινώσεις σχετικά με την ημερομηνία κυκλοφορίας της v3.', additional_kwargs={}, response_metadata={'id': 'msg_01QMG5xJNxsuGfAQTMbjZDBX', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 49, 'output_tokens': 319}}, id='run-16756bc3-c706-4f3e-a503-752217343c20-0', usage_metadata={'input_tokens': 49, 'output_tokens': 319, 'total_tokens': 368, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [31]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
  def parse(self, text):
    item =text.strip().split(",")
    return list(map(str.strip,item)) #공백도 제거

# p=CommaOutputParser()
# p.parse("Hello, How,are,u,?")

In [42]:
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked wull be answered with a comma seperted list of max {max_items}. Do not reply with anything else."),
  ("human", "{question}"),]
)

prompt = template.format(
  max_items = 10,
  question = "what are the colors?"
)

result = chat_model.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']